# تمرین سری سوم درس پردازش داده های حجیم
# زهرا دهقانیان
# 401300417




In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq --fix-missing
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 85 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u312-b07-0ubuntu1~18.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u312-b07-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jdk-headl

In [2]:
!apt-get update


0% [Working]
            
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]

0% [Connecting to archive.ubuntu.com (91.189.88.142)] [Connecting to security.u
0% [Connecting to archive.ubuntu.com (91.189.88.142)] [Connecting to security.u
0% [1 InRelease gpgv 3,626 B] [Connecting to archive.ubuntu.com (91.189.88.142)
                                                                               
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease

0% [1 InRelease gpgv 3,626 B] [Waiting for headers] [Waiting for headers] [Wait
                                                                               
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]

0% [1 InRelease gpgv 3,626 B] [Waiting for headers] [Waiting for headers] [3 In
                                                                               
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http

In [4]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .appName("Fave_Project") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [6]:
df=spark.read.csv('/content/drive/My Drive/Sample_Data/Sample_Traffic.csv',header=True,schema=schema)
df.show(1)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



الف) مسیر عبوری هر خودرو را به تفکیک روز مشخص کنید 

In [7]:
df_mapped = df.rdd.map(lambda x: ((x['ORIGINE_CAR_KEY'], x['PASS_DAY_TIME'].date()), [x['DEVICE_CODE']]))
df_mapped3 = df_mapped.reduceByKey(lambda a,b: a+b)
df_mapped2 = df_mapped3.sample(False,0.1)
df_mapped2.take(10)

[(('35898300', datetime.date(2021, 6, 1)), [631795]),
 (('97890742', datetime.date(2021, 6, 1)), [631829]),
 (('27519912', datetime.date(2021, 6, 1)),
  [145,
   900107,
   145,
   202601,
   22009977,
   900134,
   631368,
   114,
   114,
   900142,
   900236,
   900132,
   900108,
   100700814,
   22010119,
   900107,
   100700804,
   900276,
   900152,
   100700834,
   900164,
   100701090,
   900156,
   900108,
   100700824,
   209102]),
 (('61880025', datetime.date(2021, 6, 1)), [631829]),
 (('60461263', datetime.date(2021, 6, 1)), [115]),
 (('8389415', datetime.date(2021, 6, 1)), [202101, 113, 145, 900101]),
 (('35458343', datetime.date(2021, 6, 1)),
  [202101, 145, 113, 22009971, 200902, 102, 631832, 22009972, 900215]),
 (('12349968', datetime.date(2021, 6, 1)),
  [205201, 100, 631832, 103002, 900269, 22009972, 900124, 900217, 900215]),
 (('13792945', datetime.date(2021, 6, 1)),
  [631795, 900143, 900152, 900152, 900191]),
 (('8585601', datetime.date(2021, 6, 1)), [202601])]

ب)محاسبه شباهت کسینوسی بین یک مسیر فرضی و مسیر خودروها و گزارش پنج مسیر و خودرو با بیشترین شباهت  

برای یافتن فاصله‌ی کسینوسی به جای تشکیل بردار با طول هزار متشکل از مقادیر صفر و یک برای هر مسیر میتوان تعداد کدهای مشترک بین هر مسیر و مسیر فرضی را بدست آورد و سپس بر اندازه‌ی بردار مسیر پیشنهادی و همچنین اندازه‌ی مسیر هر خودرو تقسیم نمود.
 در این مساله خروجی این روش همان خروجی موردنظر یعنی شباهت کسینوسی خواهد بود و فقط بسیار ساده تر محاسبه می شود

In [45]:
import math 
path = [900142,900236,900132,900108,100700814] #masire farzi
lenPath = 10
similarity_rdd = df_mapped2.map(lambda x: ((x[0][0], x[1]), len(list(set(path) & set(x[1])))/math.sqrt(len(x[1])*lenPath)))
similarity_rdd.take(20)

[(('35898300', [631795]), 0.0),
 (('97890742', [631829]), 0.0),
 (('27519912',
   [145,
    900107,
    145,
    202601,
    22009977,
    900134,
    631368,
    114,
    114,
    900142,
    900236,
    900132,
    900108,
    100700814,
    22010119,
    900107,
    100700804,
    900276,
    900152,
    100700834,
    900164,
    100701090,
    900156,
    900108,
    100700824,
    209102]),
  0.31008683647302115),
 (('61880025', [631829]), 0.0),
 (('60461263', [115]), 0.0),
 (('8389415', [202101, 113, 145, 900101]), 0.0),
 (('35458343',
   [202101, 145, 113, 22009971, 200902, 102, 631832, 22009972, 900215]),
  0.0),
 (('12349968',
   [205201, 100, 631832, 103002, 900269, 22009972, 900124, 900217, 900215]),
  0.0),
 (('13792945', [631795, 900143, 900152, 900152, 900191]), 0.0),
 (('8585601', [202601]), 0.0),
 (('9032461', [631832, 900269, 900185, 900276]), 0.0),
 (('13916077', [208601, 208602, 206701, 117, 135, 900135]), 0.0),
 (('21302934', [112]), 0.0),
 (('9263848',
   [631829,

پس از بدست آمدن مقادیر شباهت کسینوسی هر مسیر با مسیر فرضی ۵ مسیر و خودرو با بیشترین شباهت با مسیر فرضی را نمایش میدهیم

In [46]:
similarPath = similarity_rdd.takeOrdered(5, key = lambda x: -x[1])
similarPath

[(('8862590', [900132, 900142]), 0.4472135954999579),
 (('68151032', [100700814, 900108]), 0.4472135954999579),
 (('77627646', [900236, 900142]), 0.4472135954999579),
 (('10285799', [900236, 900142]), 0.4472135954999579),
 (('46037696', [900142, 900132]), 0.4472135954999579)]

پ) با استفاده از روش ال اس اج بخش ب را حل کنید  و محاسبات مربوط به نحوه
استفاده از آن ها را انجام دهید. سپس مسیر های مشابه با مسیر فرضی را گزارش کنید. در صورت افزایش تعداد هایپرپلین ها دقت به چه صورت افزایش می یابد.

برای یافتن بردارهای مشابه با بردار فرضی تعریف شده به کمک روش ال اس اچ، اول بردار دوربینهای مجزا را می سازیم. در این مسیله ابتدا تعداد توابع مین هش یعنی هایپرپلین ها را برابر با ۱۰۰ قرار میدهیم و در ادامه تغییرات را با دوبرابر کردنشان بررسی میکنیم. 

In [8]:
allcamera = df_mapped2.flatMap(lambda line: line[1])
allcameradist = allcamera.distinct().collect()
sc.broadcast(allcameradist)

 برای یافتن بردار هش شده‌ی به طول ۱۰۰ از هر یک از مسیرها، بر اساس وجود اولین یک در هر یک از  جایگشت رندوم ایجاد شده در بالا، عمل می کنیم. در دو تابع زیر این فرآیند انجام میگیرد

In [47]:
def findplace(arr, start, size):
  result = []
  for x in arr:   
    if(x >= start):
      result.append( x - start)
    else:
      result.append( size - 1 - start + x)
  return arr[result.index(min(result))]


def minHashing(row):
  x = row[1]
  result = []
  place = []
  for j in range(len(x)):
      place.append(allcameradist.index(x[j])) 
  for i in range(100):
      result.append(findplace(place, random_start[i], len(allcameradist)))

  return (row[0], result)

import numpy as np
random_start = list(np.random.randint(0,936, size=100))
sc.broadcast(random_start)
df_min_hashing = df_mapped2.map(minHashing)
df_min_hashing.take(5)

[(('35898300', datetime.date(2021, 6, 1)),
  [320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320,
   320]),
 (('97890742', datetime.date(2021, 6, 1)),
  [625,
   625,
   625,
   625,
   625,
   625,
   625,
   625,
   625,
   625,
   625,
   625,
   625,
   625,


همین کار را برای مسیر فرضیمان نیز انجام میدهیم و بردار هش شده به طول ۱۰۰ آن را بدست می‌آوریم

In [48]:
path = (0,[900142,900236,900132,900108,100700814])
hashed_path = minHashing(path)
hashed_path

(0,
 [3,
  629,
  3,
  322,
  629,
  3,
  629,
  3,
  322,
  629,
  629,
  3,
  322,
  629,
  3,
  322,
  3,
  322,
  322,
  322,
  3,
  629,
  5,
  3,
  3,
  3,
  3,
  629,
  629,
  3,
  629,
  322,
  322,
  629,
  322,
  322,
  3,
  322,
  629,
  322,
  629,
  322,
  3,
  3,
  3,
  3,
  322,
  322,
  629,
  322,
  3,
  322,
  3,
  3,
  3,
  629,
  629,
  322,
  629,
  322,
  629,
  322,
  322,
  3,
  3,
  629,
  3,
  629,
  3,
  322,
  322,
  629,
  3,
  629,
  3,
  3,
  3,
  629,
  629,
  3,
  3,
  629,
  322,
  322,
  629,
  322,
  322,
  3,
  629,
  3,
  3,
  629,
  3,
  322,
  3,
  322,
  629,
  629,
  322,
  629])

پس از بدست آوردن بردار هش شده هر مسیر،‌هر ۲۰ سطر از سیگنچر هر مسیر را به کمک هش کردن به ۵ باکت مپ میکنیم پس در نهایت برای هر مسیر ۵ مقدار داریم که مقدار هر کدام نشاندهنده‌ی باکتی هست که به کمک هشینگ در آن قرار گرفته

In [50]:
def LSH(x):
  mhash = x[1]
  result = list()
  for i in range(5):
    result.append(abs(hash(str(mhash[i*20:(i+1)*20]))) % 5)
  return (x[0], result)
df_lsh = df_min_hashing.map(LSH)
df_lsh.take(10)

[(('35898300', datetime.date(2021, 6, 1)), [4, 4, 4, 4, 4]),
 (('97890742', datetime.date(2021, 6, 1)), [0, 0, 0, 0, 0]),
 (('27519912', datetime.date(2021, 6, 1)), [4, 4, 4, 2, 1]),
 (('61880025', datetime.date(2021, 6, 1)), [0, 0, 0, 0, 0]),
 (('60461263', datetime.date(2021, 6, 1)), [1, 1, 1, 1, 1]),
 (('8389415', datetime.date(2021, 6, 1)), [3, 3, 2, 1, 0]),
 (('35458343', datetime.date(2021, 6, 1)), [3, 3, 1, 1, 3]),
 (('12349968', datetime.date(2021, 6, 1)), [2, 2, 4, 1, 4]),
 (('13792945', datetime.date(2021, 6, 1)), [4, 3, 1, 2, 2]),
 (('8585601', datetime.date(2021, 6, 1)), [2, 2, 2, 2, 2])]

همین کار را برای مسیر فرضی نیز انجام میدهیم

In [51]:
path_LSH = LSH(hashed_path)[1]
sc.broadcast(path_LSH)
path_LSH

[4, 3, 3, 0, 0]

پس از بدست آوردن باکت های هش هر مسیر آن را به دو صورت با باکت های هش مسیر فرضی 
مقایسه میکنیم


1.   and
2.   or with similarity score

در خروجی زیر مقدار اول در مقدار هر مسیر خروجی حاصل برای اند و مقدار دوم خروجی اور و مقدار سوم تعداد باکت های یکسان تقسیم بر کل باکت ها میباشد


In [52]:
def check_similarity(x):
  y = x[1]
  orres = False
  andres = True
  sum = 0
  for i in range(len(y)):
    andres = andres and (path_LSH[i] == y[i])
    orres = orres or (path_LSH[i] == y[i])
    if(path_LSH[i] == y[i]):
      sum += 1

  return (x[0],(y,andres,orres,sum/len(y)))

df_similarity_LSH = df_lsh.map(check_similarity)
df_similarity_LSH.take(10)

[(('35898300', datetime.date(2021, 6, 1)),
  ([4, 4, 4, 4, 4], False, True, 0.2)),
 (('97890742', datetime.date(2021, 6, 1)),
  ([0, 0, 0, 0, 0], False, True, 0.4)),
 (('27519912', datetime.date(2021, 6, 1)),
  ([4, 4, 4, 2, 1], False, True, 0.2)),
 (('61880025', datetime.date(2021, 6, 1)),
  ([0, 0, 0, 0, 0], False, True, 0.4)),
 (('60461263', datetime.date(2021, 6, 1)),
  ([1, 1, 1, 1, 1], False, False, 0.0)),
 (('8389415', datetime.date(2021, 6, 1)), ([3, 3, 2, 1, 0], False, True, 0.4)),
 (('35458343', datetime.date(2021, 6, 1)),
  ([3, 3, 1, 1, 3], False, True, 0.2)),
 (('12349968', datetime.date(2021, 6, 1)),
  ([2, 2, 4, 1, 4], False, False, 0.0)),
 (('13792945', datetime.date(2021, 6, 1)),
  ([4, 3, 1, 2, 2], False, True, 0.4)),
 (('8585601', datetime.date(2021, 6, 1)),
  ([2, 2, 2, 2, 2], False, False, 0.0))]

In [53]:
similar_or = df_similarity_LSH.takeOrdered(40, key = lambda x: -x[1][3])
similar_or

[(('76970054', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('14354488', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('11412024', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('68793673', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('19344028', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('8694323', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('64897682', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('21754488', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('23773172', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('26505927', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('25469095', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('25632866', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('12397776', datetime.date(2021, 6, 1))

In [54]:
andtrue = df_similarity_LSH.filter(lambda x: x[1][1] == True)
andtrue.collect()

[(('76970054', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('14354488', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('11412024', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('68793673', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('19344028', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('8694323', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('64897682', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('21754488', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('23773172', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('26505927', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('25469095', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('25632866', datetime.date(2021, 6, 1)), ([4, 3, 3, 0, 0], True, True, 1.0)),
 (('12397776', datetime.date(2021, 6, 1))

حال همین کار را با تعداد هایپر پلین های ۲۰۰  انجام میدهیم


In [61]:
random_start200 = list(np.random.randint(0,936, size=200))
sc.broadcast(random_start200)

def findplace(arr, start, size):
  result = []
  for x in arr:   
    if(x >= start):
      result.append( x - start)
    else:
      result.append( size - 1 - start + x)
  return arr[result.index(min(result))]


def minHashing2(row):
  x = row[1]
  result = []
  place = []
  for j in range(len(x)):
      place.append(allcameradist.index(x[j])) 
  for i in range(200):
      result.append(findplace(place, random_start200[i], len(allcameradist)))

  return (row[0], result)

df_min_hashing2 = df_mapped2.map(minHashing2)

hashed_path2 = minHashing2(path)

def LSH2(x):
  mhash = x[1]
  result = list()
  for i in range(10):
    result.append(abs(hash(str(mhash[i*20:(i+1)*20]))) % 5)
  return (x[0], result)


df_lsh2 = df_min_hashing2.map(LSH2)
df_lsh2.take(10)

path_LSH2 = LSH2(hashed_path2)[1]
sc.broadcast(path_LSH2)

def check_similarity(x):
  y = x[1]
  orres = False
  andres = True
  sum = 0
  for i in range(len(y)):
    andres = andres and (path_LSH2[i] == y[i])
    orres = orres or (path_LSH2[i] == y[i])
    if(path_LSH2[i] == y[i]):
      sum += 1

  return (x[0],(y,andres,orres,sum/len(y)))

df_similarity_LSH2 = df_lsh2.map(check_similarity)
df_similarity_LSH2.take(10)

[(('35898300', datetime.date(2021, 6, 1)),
  ([4, 4, 4, 4, 4, 4, 4, 4, 4, 4], False, True, 0.2)),
 (('97890742', datetime.date(2021, 6, 1)),
  ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], False, True, 0.2)),
 (('27519912', datetime.date(2021, 6, 1)),
  ([3, 0, 0, 2, 2, 0, 2, 2, 2, 2], False, True, 0.2)),
 (('61880025', datetime.date(2021, 6, 1)),
  ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], False, True, 0.2)),
 (('60461263', datetime.date(2021, 6, 1)),
  ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], False, False, 0.0)),
 (('8389415', datetime.date(2021, 6, 1)),
  ([2, 4, 2, 1, 4, 4, 4, 1, 2, 4], False, True, 0.1)),
 (('35458343', datetime.date(2021, 6, 1)),
  ([1, 4, 2, 0, 4, 4, 4, 1, 2, 4], False, True, 0.1)),
 (('12349968', datetime.date(2021, 6, 1)),
  ([3, 0, 1, 0, 4, 0, 0, 2, 1, 4], False, True, 0.4)),
 (('13792945', datetime.date(2021, 6, 1)),
  ([0, 0, 4, 0, 3, 4, 1, 2, 0, 2], False, True, 0.1)),
 (('8585601', datetime.date(2021, 6, 1)),
  ([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], False, True, 0.1))]

In [63]:
similar_or2 = df_similarity_LSH2.takeOrdered(40, key = lambda x: -x[1][3])
similar_or2

[(('9525774', datetime.date(2021, 6, 1)),
  ([3, 2, 4, 3, 3, 1, 0, 3, 4, 4], False, True, 0.8)),
 (('90923203', datetime.date(2021, 6, 1)),
  ([3, 0, 3, 3, 3, 4, 0, 3, 4, 4], False, True, 0.8)),
 (('9457910', datetime.date(2021, 6, 1)),
  ([3, 3, 3, 3, 3, 0, 0, 3, 3, 4], False, True, 0.8)),
 (('98666508', datetime.date(2021, 6, 1)),
  ([3, 0, 3, 3, 3, 4, 0, 3, 4, 4], False, True, 0.8)),
 (('40566254', datetime.date(2021, 6, 1)),
  ([3, 0, 3, 3, 3, 4, 0, 3, 4, 4], False, True, 0.8)),
 (('97844749', datetime.date(2021, 6, 1)),
  ([3, 2, 3, 2, 3, 3, 0, 3, 4, 4], False, True, 0.8)),
 (('54397550', datetime.date(2021, 6, 1)),
  ([3, 0, 3, 3, 3, 4, 0, 3, 4, 4], False, True, 0.8)),
 (('57626967', datetime.date(2021, 6, 1)),
  ([3, 0, 3, 3, 3, 4, 0, 3, 4, 4], False, True, 0.8)),
 (('10478887', datetime.date(2021, 6, 1)),
  ([3, 2, 3, 3, 3, 0, 0, 1, 0, 4], False, True, 0.8)),
 (('10092844', datetime.date(2021, 6, 1)),
  ([3, 0, 3, 3, 3, 4, 0, 3, 4, 4], False, True, 0.8)),
 (('9930822', datetime

بنظر می رسد با افزایش تعداد هایپر پلین ها، دقت به صورت نمایی بهتر شده است  